# Taxi Trajectory

#### We have provided with the dataset describing a complete year (from 1 july 2013 to 30 june 2014) of the trajectories for all the 442 taxies running in the city of portugal.
#### We are hired by the transportation industry of portugal and want to predict the total travel time of the trip based on the attributes given in the datasets.

## Data Overview

Each data point corresponds to one complete trip. It contains a total 9 features

* TRIP_ID (string) : It contains an unique identifier for each trip
* CALL_TYPE (char) :  It identifies the way used to demand this service. It may contain one of three possible values.
                      1. 'A' : if this trip was dispatched from the central
                      2. 'B' : if this trip was demanded directly to a taxi driver on a specific stand
                      3. 'C' : otherwise (i.e. a trip demanded on a random street)
* ORIGIN_CALL (integer) :  It contains an unique identifier for each phone number which was used to demand, at least, one service. It identifies the trip’s customer if CALL_TYPE=’A’. Otherwise, it assumes a NULL value
* ORIGIN_STAND (integer): It contains an unique identifier for the taxi stand. It identifies the starting point of the trip if CALL_TYPE=’B’. Otherwise, it assumes a NULL value
* TAXI_ID: (integer): It contains an unique identifier for the taxi driver that performed each trip
* TIMESTAMP (integer) : Unix Timestamp (in seconds). It identifies the trip’s start
* DAYTYPE (char) : It identifies the daytype of the trip’s start. It assumes one of three possible values
                    1. 'B' : if this trip started on a holiday or any other special day (i.e. extending holidays, floating holidays, etc.)
                    2. 'C' : if the trip started on a day before a type-B day
                    3. 'A' : otherwise (i.e. a normal day, workday or weekend)
* MISSING_DATA (Boolean) : It is FALSE when the GPS data stream is complete and TRUE whenever one (or more) locations are missing.
* POLYLINE (String): It contains a list of GPS coordinates (i.e. WGS84 format) mapped as a string.The last list item corresponds to the trip’s destination while the first one represents its start. The beginning and the end of the string are identified with brackets (i.e. [ and ], respectively). Each pair of coordinates is also identified by the same brackets as [LONGITUDE, LATITUDE]. This list contains one pair of coordinates for each 15 seconds of trip. The POLYLINE can have multiple pairs of longitude and latitude


The total travel time of the trip (the prediction target) is defined as the (number of points-1) x 15 seconds. For example, a trip with 101 data points in POLYLINE has a length of (101-1) * 15 = 1500 seconds.




##### Type of Machine Learning task :
It is an regression problem where given a set of features we need to predict the total travel time by taxi from starting of ride to the destination in seconds.
                    
                

In [1]:
# Install required packages if not already installed
# Using !pip works better in Jupyter notebooks
import sys

def check_and_install(package, import_name=None):
    if import_name is None:
        import_name = package
    try:
        __import__(import_name)
        print(f"✓ {package} is already installed")
        return True
    except ImportError:
        print(f"Installing {package}...")
        import subprocess
        result = subprocess.run([sys.executable, "-m", "pip", "install", package, "--quiet"], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            print(f"✓ {package} installed successfully!")
            print("⚠ NOTE: Please RESTART THE KERNEL after this cell runs, then run all cells again.")
            print("   Go to: Kernel → Restart Kernel → Run All")
            return True
        else:
            print(f"✗ Failed to install {package}")
            return False

# Check and install missing packages
print("Checking required packages...")
check_and_install('xgboost')
check_and_install('mlxtend')
check_and_install('seaborn', 'seaborn')

print("\n" + "="*60)
print("IMPORTANT: After packages are installed, RESTART THE KERNEL")
print("Then run all cells again (Cell → Run All)")
print("="*60)


Checking required packages...
✓ xgboost is already installed
✓ mlxtend is already installed
✓ seaborn is already installed

IMPORTANT: After packages are installed, RESTART THE KERNEL
Then run all cells again (Cell → Run All)


#### Performace Metric
Since it is an regression problem we will use Root Mean Squared error (RMSE) and R-squared as regression metric.

#### Importing Libraries

In [2]:
# IMPORTANT: If you get ModuleNotFoundError above, restart the kernel and run all cells
# After installing packages, Python needs to reload its module cache
# This cell will re-import the packages if they were just installed

try:
    from xgboost import XGBRegressor
    from mlxtend.regressor import StackingRegressor
    print("✓ xgboost and mlxtend are already imported")
except ImportError as e:
    print(f"⚠ Import error: {e}")
    print("Please restart the kernel (Kernel → Restart Kernel) and run all cells again.")
    print("Or run this command in a new cell: !pip install xgboost mlxtend")


✓ xgboost and mlxtend are already imported


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

from math import sqrt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor

import joblib

### EDA, Data Cleaning and Feature Engineering

In [4]:
# Data loading - using local file path
# The dataset file should be in the same directory as this notebook

#### Load the taxi trajectory data from CSV file

In [5]:
# Read CSV with error handling for malformed rows
# Using Python engine to handle parsing errors more gracefully
print("Loading data...")
try:
    # For pandas >= 1.3.0
    df = pd.read_csv('train(1).csv', on_bad_lines='skip', engine='python')
except TypeError:
    # For pandas < 1.3.0
    try:
        df = pd.read_csv('train(1).csv', error_bad_lines=False, warn_bad_lines=False, engine='python')
    except:
        # Last resort: read in chunks and skip bad lines
        chunks = []
        for chunk in pd.read_csv('train(1).csv', chunksize=10000, engine='python', error_bad_lines=False, warn_bad_lines=False):
            chunks.append(chunk)
        df = pd.concat(chunks, ignore_index=True)

print(f"Successfully loaded {len(df)} rows")

# OPTIMIZATION: Sample for faster processing while reducing bias
# If dataset is large, take a random 50,000 row sample (shuffled) to avoid time-based skew
if len(df) > 50000:
    print("\n⚠ Sampling 50,000 rows for faster processing (random sample)...")
    df = df.sample(n=50000, random_state=42).reset_index(drop=True)
else:
    print("\nDataset has <= 50,000 rows; using all rows")
print(f"Working with {len(df)} rows")

Loading data...
Successfully loaded 325755 rows

⚠ Sampling 50,000 rows for faster processing (random sample)...
Working with 50000 rows


In [6]:
df.shape

(50000, 9)

The dataset has around 171K instances each of which has 9 different feature values.

In [7]:
df.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE'],
      dtype='object')

Displying the first 10 rows.

In [8]:
df.head(10)

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
0,1372795478620000146,C,NaN,NaN,20000146,1372795478,A,False,"[[-8.584308,41.163192],[-8.584479,41.163093],[..."
1,1375716492620000166,B,NaN,27.0,20000166,1375716492,A,False,"[[-8.608698,41.147811],[-8.608707,41.147811],[..."
2,1374123797620000325,A,6584.0,NaN,20000325,1374123797,A,False,"[[-8.611641,41.149287],[-8.611956,41.149125],[..."
3,1378513889620000560,B,NaN,15.0,20000560,1378513889,A,False,"[[-8.585685,41.148621],[-8.586054,41.148972],[..."
4,1378059671620000051,A,59622.0,NaN,20000051,1378059671,A,False,"[[-8.601129,41.18382],[-8.601246,41.18499],[-8..."
5,1374807707620000012,C,NaN,NaN,20000012,1374807707,A,False,"[[-8.590509,41.162751],[-8.590077,41.163309],[..."
6,1376156397620000004,B,NaN,23.0,20000004,1376156397,A,False,"[[-8.612784,41.146065],[-8.612793,41.146047]]"
7,1373068920620000688,C,NaN,NaN,20000688,1373068920,A,False,"[[-8.614935,41.149305],[-8.614845,41.149467],[..."
8,1374921050620000546,B,NaN,20.0,20000546,1374921050,A,False,"[[-8.627112,41.166909],[-8.627634,41.166999],[..."
9,1374981001620000013,C,NaN,NaN,20000013,1374981001,A,False,"[[-8.576046,41.144058],[-8.578053,41.145174],[..."


Filtering out categorical features.

In [9]:
df.dtypes[df.dtypes == 'object']

CALL_TYPE    object
DAY_TYPE     object
POLYLINE     object
dtype: object

In [10]:
df.isnull().sum()

TRIP_ID             0
CALL_TYPE           0
ORIGIN_CALL     39805
ORIGIN_STAND    24896
TAXI_ID             0
TIMESTAMP           0
DAY_TYPE            0
MISSING_DATA        0
POLYLINE            0
dtype: int64

There are many missing values in ORIGIN_CALL and ORIGIN_STAND because may be all the taxi users have not called the via phone and they have not started their trip from taxi stand.

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   TRIP_ID       50000 non-null  int64  
 1   CALL_TYPE     50000 non-null  object 
 2   ORIGIN_CALL   10195 non-null  float64
 3   ORIGIN_STAND  25104 non-null  float64
 4   TAXI_ID       50000 non-null  int64  
 5   TIMESTAMP     50000 non-null  int64  
 6   DAY_TYPE      50000 non-null  object 
 7   MISSING_DATA  50000 non-null  bool   
 8   POLYLINE      50000 non-null  object 
dtypes: bool(1), float64(2), int64(3), object(3)
memory usage: 3.1+ MB


In [12]:
df.describe()

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP
count,5.000000e+04,10195.000000,25104.000000,5.000000e+04,5.000000e+04
mean,1.375756e+18,23956.722609,30.611536,2.000035e+07,1.375756e+09
std,1.889465e+15,19781.559671,17.850738,2.097728e+02,1.889465e+06
min,1.372637e+18,2001.000000,1.000000,2.000000e+07,1.372637e+09
25%,1.374112e+18,5759.000000,15.000000,2.000017e+07,1.374112e+09
50%,1.375680e+18,18004.000000,27.000000,2.000035e+07,1.375680e+09
75%,1.377410e+18,40510.000000,50.000000,2.000052e+07,1.377410e+09
max,1.379055e+18,63882.000000,63.000000,2.000098e+07,1.379055e+09


Describing the categorical features.

In [13]:
df.describe(include = ['object'])

,CALL_TYPE,DAY_TYPE,POLYLINE
count,50000,50000,50000
unique,3,1,49830
top,B,A,[]
freq,25337,50000,169


We can see the DAY_TYPE has only 1 unique value and that is 'A' which means that all the trips are started on normal day or weekend. Also the 5901 observations don't have the POLYLINE values means we cannot calculate the travel time for those trips.

##### Sorting the entire dataset based on the timestamp

In [14]:
df.sort_values('TIMESTAMP',inplace = True)

In [15]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE
37615,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[..."
7855,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-..."
2044,1372637299620000011,C,NaN,NaN,20000011,1372637299,A,False,"[[-8.617563,41.146182],[-8.617527,41.145849],[..."
11445,1372637658620000596,A,22864.0,NaN,20000596,1372637658,A,False,"[[-8.665686,41.170626],[-8.665677,41.170653],[..."
40253,1372637728620000671,C,NaN,NaN,20000671,1372637728,A,False,"[[-8.617041,41.150367],[-8.616879,41.15007],[-..."


In [16]:
df['year'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).year)
df['month'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).month)
df['month_day'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).day)
df['hour'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).hour)
df['week_day'] = df['TIMESTAMP'].apply(lambda x :datetime.datetime.fromtimestamp(x).weekday())

In [17]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,year,month,month_day,hour,week_day
37615,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",2013,6,30,20,6
7855,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",2013,6,30,20,6
2044,1372637299620000011,C,NaN,NaN,20000011,1372637299,A,False,"[[-8.617563,41.146182],[-8.617527,41.145849],[...",2013,6,30,20,6
11445,1372637658620000596,A,22864.0,NaN,20000596,1372637658,A,False,"[[-8.665686,41.170626],[-8.665677,41.170653],[...",2013,6,30,20,6
40253,1372637728620000671,C,NaN,NaN,20000671,1372637728,A,False,"[[-8.617041,41.150367],[-8.616879,41.15007],[-...",2013,6,30,20,6


Pie chart for the year

In [18]:
plt.figure(figsize = (10,10))
plt.pie(df['year'].value_counts(), labels = df['year'].value_counts().keys(),autopct = '%.1f%%')

([<matplotlib.patches.Wedge at 0x268e24c78c0>],
 [Text(-1.0999999999999959, -9.616505800409723e-08, '2013')],
 [Text(-0.5999999999999978, -5.2453668002234845e-08, '100.0%')])

From the above pie chart it is clear that there are equal number of taxi trips in both the year.

In [19]:
plt.figure(figsize = (5,5))
plt.title('Count of trips per day of week')
sns.countplot(y = 'week_day', data = df)
plt.xlabel('Count')
plt.ylabel('Day')

Text(0, 0.5, 'Day')

The 4th and 5th day of week has almost same number of trips and rest all the days have almost similar number of trips. This means that we can say each and every day of week required same number of taxies irrespective of weekend or working day.

In [20]:
plt.figure(figsize = (10,10))
plt.title('Count of trips per month')
sns.countplot(y = 'month', data = df)
plt.xlabel('Count')
plt.ylabel('Month')

Text(0, 0.5, 'Month')

On an average we can say that every month has atleast 120000 taxi trips planned.

In [21]:
plt.figure(figsize = (10,10))
plt.title('Count of trips per hour')
sns.countplot(y = 'hour', data = df)
plt.xlabel('Count')
plt.ylabel('Hours')

Text(0, 0.5, 'Hours')

14th and 15th hour may be the peak hours, office time, school time because lot of taxies are used between this time.

In [22]:
df['MISSING_DATA'].value_counts()

MISSING_DATA
False    50000
Name: count, dtype: int64

For 10 instances the GPS data steam is not complete and there can be one or more locations missing. Such a data points wont gives us the appropriate trip time so we can drop such observations.

In [23]:
df.drop(df[df['MISSING_DATA'] == True].index, inplace = True)

In [24]:
df['MISSING_DATA'].unique()

array([False])

Also some of the POLYLINES values are missing in which we cannot find the trip time, dropping such observations is also the good idea.

In [25]:
df[df['POLYLINE'] =='[]']['POLYLINE'].value_counts()

POLYLINE
[]    169
Name: count, dtype: int64

In [26]:
df.drop(df[df['POLYLINE'] =='[]']['POLYLINE'].index, inplace = True)

In [27]:
df[df['POLYLINE'] =='[]']['POLYLINE'].value_counts()

Series([], Name: count, dtype: int64)

#### Convreting a POLYLINE into the total travelling time.

In [28]:
# OPTIMIZED: Replace slow eval() with fast ast.literal_eval() or string counting
# Method 1: Count brackets (fastest - no parsing needed)
import ast

def get_polyline_length(polyline_str):
    """Fast method to get polyline length without parsing the entire list"""
    if pd.isna(polyline_str) or polyline_str == '[]' or polyline_str == '':
        return 0
    try:
        # Count the number of coordinate pairs by counting '[' after the first one
        # Each coordinate pair is [lon, lat], so count internal brackets
        return polyline_str.count('[') - 1  # Subtract 1 for the outer brackets
    except:
        return 0

print("Calculating polyline lengths (optimized method)...")
df['Polyline Length'] = df['POLYLINE'].apply(get_polyline_length)
print("✓ Polyline lengths calculated!")

Calculating polyline lengths (optimized method)...
✓ Polyline lengths calculated!


In [29]:
# Calculate trip time directly (vectorized operation - much faster)
df['Trip Time(sec)'] = df['Polyline Length'] * 15
print("✓ Trip times calculated!")

✓ Trip times calculated!


In [30]:
df.head()

,TRIP_ID,CALL_TYPE,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,DAY_TYPE,MISSING_DATA,POLYLINE,year,month,month_day,hour,week_day,Polyline Length,Trip Time(sec)
37615,1372636858620000589,C,NaN,NaN,20000589,1372636858,A,False,"[[-8.618643,41.141412],[-8.618499,41.141376],[...",2013,6,30,20,6,23,345
7855,1372636951620000320,C,NaN,NaN,20000320,1372636951,A,False,"[[-8.612964,41.140359],[-8.613378,41.14035],[-...",2013,6,30,20,6,65,975
2044,1372637299620000011,C,NaN,NaN,20000011,1372637299,A,False,"[[-8.617563,41.146182],[-8.617527,41.145849],[...",2013,6,30,20,6,34,510
11445,1372637658620000596,A,22864.0,NaN,20000596,1372637658,A,False,"[[-8.665686,41.170626],[-8.665677,41.170653],[...",2013,6,30,20,6,26,390
40253,1372637728620000671,C,NaN,NaN,20000671,1372637728,A,False,"[[-8.617041,41.150367],[-8.616879,41.15007],[-...",2013,6,30,20,6,34,510


In [31]:
df['Trip Time(sec)'].describe()

count    49831.000000
mean       701.770384
std        609.489619
min         15.000000
25%        405.000000
50%        600.000000
75%        840.000000
max      27855.000000
Name: Trip Time(sec), dtype: float64

Above description clear that the minimum travelling time by the taxi is 15 seconds and maximum is 58200 seconds i.e.around 16 hours 16 min.

In [32]:
# Use numeric-only columns to avoid TypeError on object data
numeric_cols = df.select_dtypes(include=[np.number]).columns
week_day_mean = df.groupby('week_day')[numeric_cols].mean()
week_day_mean

,TRIP_ID,ORIGIN_CALL,ORIGIN_STAND,TAXI_ID,TIMESTAMP,year,month,month_day,hour,week_day,Polyline Length,Trip Time(sec)
week_day,,,,,,,,,,,,
0,1.375593e+18,23370.783985,30.490880,2.000035e+07,1.375593e+09,2013.0,7.692947,13.115145,10.322572,0.0,48.036303,720.544548
1,1.375785e+18,23399.398199,30.758808,2.000035e+07,1.375785e+09,2013.0,7.723285,14.404107,10.257019,1.0,46.835033,702.525492
2,1.375832e+18,24189.473248,30.612307,2.000035e+07,1.375832e+09,2013.0,7.721136,14.995641,10.636063,2.0,47.226948,708.404227
3,1.375892e+18,23955.193117,30.614877,2.000035e+07,1.375892e+09,2013.0,7.795661,13.389663,10.407472,3.0,47.935994,719.039904
4,1.375638e+18,23484.069479,30.373575,2.000035e+07,1.375638e+09,2013.0,7.669274,14.300660,12.045099,4.0,47.702762,715.541432
5,1.375758e+18,25717.184831,31.022783,2.000035e+07,1.375758e+09,2013.0,7.679282,15.379591,12.037847,5.0,45.053967,675.809504
6,1.375828e+18,23682.945098,30.244690,2.000036e+07,1.375828e+09,2013.0,7.751029,14.031955,10.156832,6.0,43.549304,653.239561


On an average each day of week 648 to 750 seconds of journey were travelled.

In [33]:
df['DAY_TYPE'].isnull().sum()

np.int64(0)

#### One Hot Encoding for Call Type

In [34]:
df = pd.get_dummies(df, columns=['CALL_TYPE'])

#### Dropping the duplicates

In [35]:
df.shape

(49831, 18)

In [36]:
df = df.drop_duplicates()
print(df.shape)

(49831, 18)


There were 2 duplicate rows which was dropped.

#### Saving the final dataframe for future use.

In [37]:
df.to_csv('Cleaned_data.csv', index = None)

#### Data Preparation for ML models.

In [38]:
# Data already sampled to 50,000 rows earlier (see loading cell)
print(f"Dataset size for ML: {len(df)} rows")

Dataset size for ML: 49831 rows


In [39]:
df.shape

(49831, 18)

In [40]:
X = df[['Polyline Length', 'CALL_TYPE_A', 'CALL_TYPE_B', 'CALL_TYPE_C']]
y = df['Trip Time(sec)']

#### Data Standardization

In [41]:
s = StandardScaler()
X = s.fit_transform(X)

In [42]:
print(np.mean(X))
np.std(X)

1.4972002820495378e-18


np.float64(1.0)

#### Train and Test splits : 70-30

In [43]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = 0.3)

In [44]:
print("The size of training input is", X_train.shape)
print("The size of training output is", y_train.shape)
print(50 *'*')
print("The size of testing input is", X_test.shape)
print("The size of testing output is", y_test.shape)

The size of training input is (34881, 4)
The size of training output is (34881,)
**************************************************
The size of testing input is (14950, 4)
The size of testing output is (14950,)


### Machine Learning Models

#### 1. Baseline Model
* In baseline model the predicted trip time would be simply the average of all trip time.
* We will use this baseline model to perform hypothesis testing for other ML complex models.

In [45]:
y_train_pred = np.ones(X_train.shape[0]) * y_train.mean() #Predicting the train results

In [46]:
y_test_pred = np.ones(y_test.shape[0]) * y_train.mean() #Predicting the test results

In [47]:
print("Train Results for Baseline Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Baseline Model:
--------------------------------------------------
Root mean squared error:  562.5410929002843
R-squared:  0.0


In [48]:
print("Test Results for Baseline Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Baseline Model:
--------------------------------------------------
Root mean squared error:  707.0158925920966
R-squared:  -0.00023595457874914239


#### 2. KNN Regressor

In [49]:
k_range  =list(range(1,30))
param =dict(n_neighbors =k_range)
knn_regressor =GridSearchCV(KNeighborsRegressor(),param,cv =10)
knn_regressor.fit(X_train,y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",KNeighborsRegressor()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'n_neighbors': [1, 2, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",10
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter in

In [50]:
print(knn_regressor.best_estimator_)
knn_regressor.best_params_

KNeighborsRegressor(n_neighbors=1)


{'n_neighbors': 1}

In [51]:
y_train_pred =knn_regressor.predict(X_train) ##Predict train result
y_test_pred =knn_regressor.predict(X_test) ##Predict test result

In [52]:
print("Train Results for KNN Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for KNN Regressor Model:
--------------------------------------------------
Root mean squared error:  0.0
R-squared:  1.0


In [53]:
print("Test Results for KNN Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for KNN Regressor Model:
--------------------------------------------------
Root mean squared error:  75.93362045707464
R-squared:  0.98846248380156


We can see that the training RMSE error is low but testing RMSE error is high which means that model is overfitting.

#### 3. Ridge Regressor

In [54]:
params ={'alpha' :[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
ridge_regressor =GridSearchCV(Ridge(), params ,cv =5,scoring = 'neg_mean_absolute_error', n_jobs =-1)
ridge_regressor.fit(X_train ,y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Ridge()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'alpha': [0.0001, 0.001, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_absolute_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",5
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate param

In [55]:
print(ridge_regressor.best_estimator_)
ridge_regressor.best_params_

Ridge(alpha=0.0001)


{'alpha': 0.0001}

In [56]:
y_train_pred =ridge_regressor.predict(X_train) ##Predict train result
y_test_pred =ridge_regressor.predict(X_test) ##Predict test result

In [57]:
print("Train Results for Ridge Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Ridge Regressor Model:
--------------------------------------------------
Root mean squared error:  1.9028695260706584e-06
R-squared:  1.0


In [58]:
print("Test Results for Ridge Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Ridge Regressor Model:
--------------------------------------------------
Root mean squared error:  2.3955241236974106e-06
R-squared:  1.0


#### 4. Lasso Regression

In [59]:
params ={'alpha' :[0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000, 100000]}
lasso_regressor =GridSearchCV(Lasso(), params ,cv =15,scoring = 'neg_mean_absolute_error', n_jobs =-1)
lasso_regressor.fit(X_train ,y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",Lasso()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'alpha': [0.0001, 0.001, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",'neg_mean_absolute_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",15
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate para

In [60]:
print(lasso_regressor.best_estimator_)
lasso_regressor.best_params_

Lasso(alpha=0.0001)


{'alpha': 0.0001}

In [61]:
y_train_pred =lasso_regressor.predict(X_train) ##Predict train result
y_test_pred =lasso_regressor.predict(X_test) ##Predict test result

In [62]:
print("Train Results for Lasso Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Lasso Regressor Model:
--------------------------------------------------
Root mean squared error:  0.00010834470790002796
R-squared:  0.9999999999999629


In [63]:
print("Test Results for Lasso Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Lasso Regressor Model:
--------------------------------------------------
Root mean squared error:  0.00013617037285993248
R-squared:  0.9999999999999629


#### 5. Decision Tree Regressor

In [64]:
depth  =list(range(3,30))
param_grid =dict(max_depth =depth)
tree =GridSearchCV(DecisionTreeRegressor(),param_grid,cv =10)
tree.fit(X_train,y_train)

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",DecisionTreeRegressor()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'max_depth': [3, 4, ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",None
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",10
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 : the fold and candidate parameter in

In [65]:
print(tree.best_estimator_)
tree.best_params_

DecisionTreeRegressor(max_depth=26)


{'max_depth': 26}

In [66]:
y_train_pred =tree.predict(X_train) ##Predict train result
y_test_pred =tree.predict(X_test) ##Predict test result

In [67]:
print("Train Results for Decision Tree Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Decision Tree Regressor Model:
--------------------------------------------------
Root mean squared error:  0.0
R-squared:  1.0


In [68]:
print("Test Results for Decision Tree Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Decision Tree Regressor Model:
--------------------------------------------------
Root mean squared error:  77.23057602342287
R-squared:  0.9880649935668167


#### 6. XGBoost

In [69]:
tuned_params = {'max_depth': [1, 2, 3, 4, 5], 'learning_rate': [0.01, 0.05, 0.1], 'n_estimators': [100, 200, 300, 400, 500], 'reg_lambda': [0.001, 0.1, 1.0, 10.0, 100.0]}
model = RandomizedSearchCV(XGBRegressor(), tuned_params, n_iter=20, scoring = 'neg_mean_absolute_error', cv=5, n_jobs=-1)
model.fit(X_train, y_train)

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.","XGBRegressor(...ree=None, ...)"
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'learning_rate': [0.01, 0.05, ...], 'max_depth': [1, 2, ...], 'n_estimators': [100, 200, ...], 'reg_lambda': [0.001, 0.1, ...]}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",20
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'neg_mean_absolute_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that ca

In [70]:
print(model.best_estimator_)
model.best_params_

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=None, num_parallel_tree=None, ...)


{'reg_lambda': 0.1, 'n_estimators': 400, 'max_depth': 5, 'learning_rate': 0.05}

In [71]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [72]:
print("Train Results for XGBoost Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for XGBoost Regressor Model:
--------------------------------------------------
Root mean squared error:  129.39496373748477
R-squared:  0.9470914006233215


In [73]:
print("Test Results for XGBoost Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for XGBoost Regressor Model:
--------------------------------------------------
Root mean squared error:  216.86237355175285
R-squared:  0.9058950543403625


#### 7. Random Forest Regressor

In [74]:
tuned_params = {'n_estimators': [100, 200, 300, 400, 500], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}
random_regressor = RandomizedSearchCV(RandomForestRegressor(), tuned_params, n_iter = 20, scoring = 'neg_mean_absolute_error', cv = 5, n_jobs = -1)
random_regressor.fit(X_train, y_train)

,"estimator estimator: estimator objectAn object of that type is instantiated for each grid point.This is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",RandomForestRegressor()
,"param_distributions param_distributions: dict or list of dictsDictionary with parameters names (`str`) as keys and distributionsor lists of parameters to try. Distributions must provide a ``rvs``method for sampling (such as those from scipy.stats.distributions).If a list is given, it is sampled uniformly.If a list of dicts is given, first a dict is sampled uniformly, andthen a parameter is sampled using that dict as above.","{'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...], 'n_estimators': [100, 200, ...]}"
,"n_iter n_iter: int, default=10Number of parameter settings that are sampled. n_iter tradesoff runtime vs quality of the solution.",20
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.If None, the estimator's score method is used.",'neg_mean_absolute_error'
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given the ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``RandomizedSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.

In [75]:
print(random_regressor.best_estimator_)
random_regressor.best_params_

RandomForestRegressor()


{'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 1}

In [76]:
y_train_pred = random_regressor.predict(X_train)
y_test_pred = random_regressor.predict(X_test)

In [77]:
print("Train Results for Random Forest Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Random Forest Regressor Model:
--------------------------------------------------
Root mean squared error:  9.414251853161804
R-squared:  0.9997199322387416


In [78]:
print("Test Results for Random Forest Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Random Forest Regressor Model:
--------------------------------------------------
Root mean squared error:  93.54983177653533
R-squared:  0.98248822650944


#### 8. Stacking

In stacking method we stack many different ML models to get the better results.

In [79]:
# Initializing models
ridge = Ridge()
lasso = Lasso()
tree = DecisionTreeRegressor()
knn = KNeighborsRegressor()

stack = StackingRegressor(regressors = [ridge, lasso, knn], meta_regressor = tree)
stack.fit(X_train, y_train)

C:\Users\dhanu\AppData\Roaming\Python\Python313\site-packages\sklearn\externals\_numpydoc\docscrape.py:203: UserWarning: potentially wrong underline length... 
Examples 
----------- in 
A Stacking regressor for scikit-learn estimators for regression.
...
  while not self._is_at_section() and not self._doc.eof():


,regressors,"[Ridge(), Lasso(), ...]"
,meta_regressor,DecisionTreeRegressor()
,verbose,0
,use_features_in_secondary,False
,store_train_meta_features,False
,refit,True
,multi_output,False
,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2


In [80]:
print(stack.regr_)
stack.meta_regr_

[Ridge(), Lasso(), KNeighborsRegressor()]


,"criterion criterion: {""squared_error"", ""friedman_mse"", ""absolute_error"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in the half mean Poisson deviance to find splits... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 0.24 Poisson deviance criterion.",'squared_error'
,"splitter splitter: {""best"", ""random""}, default=""best""The strategy used to choose the split at each node. Supportedstrategies are ""best"" to choose the best split and ""random"" to choosethe best random split.",'best'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.For an example of how ``max_depth`` influences the model, see:ref:`sphx_glr_auto_examples_tree_plot_tree_regression.py`.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: int, float or {""sqrt"", ""log2""}, default=NoneThe number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None, then `max_features=n_features`.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",None
,"random_state random_state: int, RandomState instance or None, default=NoneControls the randomness of the estimator. The features are alwaysrandomly permuted at each split, even if ``splitter`` is set to``""best""``. When ``max_features < n_features``, the algorithm willselect ``max_features`` at random at each split before finding the bestsplit among them. But the best found split may vary across differentruns, even if ``max_features=n_features``. That is the case, if theimprovement of the criterion is identical for several splits and onesplit has to be selected at random. To obtain a deterministic behaviourduring fitting, ``random_state`` has to be fixed to an integer.See :term:`Glossary ` for details.",None
,"max

In [81]:
y_train_pred = stack.predict(X_train)
y_test_pred = stack.predict(X_test)

In [82]:
print("Train Results for Stacking Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_train.values, y_train_pred)))
print("R-squared: ", r2_score(y_train.values, y_train_pred))

Train Results for Stacking Regressor Model:
--------------------------------------------------
Root mean squared error:  0.0
R-squared:  1.0


In [83]:
print("Test Results for Stacking Regressor Model:")
print(50 * '-')
print("Root mean squared error: ", sqrt(mse(y_test, y_test_pred)))
print("R-squared: ", r2_score(y_test, y_test_pred))

Test Results for Stacking Regressor Model:
--------------------------------------------------
Root mean squared error:  75.8047791929703
R-squared:  0.9885016034178136


From the above all the models that we have trained most of them are overfitted including some ensemble teachniques also. But the random forest algorithm gives pretty good results. So the random forest is the best suited model for this dataset.

#### Save the winnig model to disk

In [84]:
win_model = RandomForestRegressor(n_estimators = 200, min_samples_split = 2, min_samples_leaf = 1)
win_model.fit(X_train, y_train)
joblib.dump(win_model, 'winnig_model_random_forest.pkl')

['winnig_model_random_forest.pkl']